In [ ]:
from src.third_party.demofusion.pipeline_highres_sdxl import StableDiffusionXLHighResPipeline
# from src.third_party.demofusion.pipeline_demofusion_sdxl import DemoFusionSDXLPipeline
import torch

from src.linfusion import LinFusion
from src.tools import seed_everything

In [ ]:
model_ckpt = "stabilityai/stable-diffusion-xl-base-1.0"
device = torch.device('cuda')
pipe = StableDiffusionXLHighResPipeline.from_pretrained(model_ckpt, torch_dtype=torch.float16, variant='fp16', local_files_only=True)
# pipe = DemoFusionSDXLPipeline.from_pretrained(model_ckpt, torch_dtype=torch.float16, variant='fp16', local_files_only=True)
pipe = pipe.to(device)

In [ ]:
linfusion = LinFusion.construct_for(pipe)

In [ ]:
prompt = "An astronaut floating in space. Beautiful view of the stars and the universe in the background."
# prompt = "A photo of the Milky Way galaxy"
generator = torch.manual_seed(42)
pipe.enable_vae_tiling()
# prompt = "Envision a portrait of an elderly woman, her face a canvas of time, framed by a headscarf with muted tones of rust and cream. Her eyes, blue like faded denim. Her attire, simple yet dignified."
# negative_prompt = "blurry, ugly, duplicate, poorly drawn, deformed, mosaic"
images = pipe(prompt, #negative_prompt=negative_prompt,
              height=1024, width=2048, device=device, view_batch_size=16, stride=64,
              num_inference_steps=50, guidance_scale=7.5,
              cosine_scale_1=3, cosine_scale_2=1, cosine_scale_3=1, gaussian_sigma=0.8,
              multi_decoder=True, show_image=True, generator=generator, upscale_strength=0.32)

In [ ]:
pipe._execution_device

In [ ]:
from diffusers import StableDiffusionXLImg2ImgPipeline

In [ ]:
model_ckpt = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(model_ckpt, torch_dtype=torch.float16, variant='fp16', local_files_only=True)
pipe = pipe.to("cuda")

In [ ]:
prompt = "An astronaut floating in space. Beautiful view of the stars and the universe in the background."
generator = torch.manual_seed(42)
pipe.enable_vae_tiling()
# prompt = "Envision a portrait of an elderly woman, her face a canvas of time, framed by a headscarf with muted tones of rust and cream. Her eyes, blue like faded denim. Her attire, simple yet dignified."
negative_prompt = "blurry, ugly, duplicate, poorly drawn, deformed, mosaic"
images_ = pipe(prompt, #negative_prompt=negative_prompt,
              image=images.images[0][0].resize((2048, 2048)),
              num_inference_steps=50, guidance_scale=7.5,
              generator=generator, strength=0.32).images
images_[0]